<a href="https://colab.research.google.com/github/nafis-momeni/BioRED_LLM/blob/main/Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-cloud-aiplatform

In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=RkdvT11gx2KgNjJfjAoM4R5Tjfa2ew&prompt=consent&token_usage=remote&access_type=offline&code_challenge=7JOsQWcaiZkk9efopgeO4OYbJNj8Kj95-S9WzoGSGR8&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0ATx3LY4dWhDldFTOl-B8rpavB48iZ98VG4yP4wBXoEQ3waOOm1boaXZodELrhyqdoY9cYA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [3]:
!gcloud auth application-default set-quota-project solid-mantra-422614-h6


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "solid-mantra-422614-h6" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [4]:
# Install the client library and import necessary modules.
# import google.generativeai as genai
import time
import json
import requests
import random

import IPython.display
from IPython.display import Markdown



In [5]:
with open('/content/new_test.json') as file:
    new_test = json.load(file)
# with open('/content/new_dev.json') as file:
#     new_dev = json.load(file)
# with open('/content/new_train.json') as file:
#     new_train = json.load(file)

In [6]:
# @title init prompt
init_prompt = '''You are an AI assistant specializing in extracting biomedical relations from scientific articles. Your task is to identify relevant relations between biomedical entities in the given text and output them in a structured format.

Input Format:
1. Title of the article
2. Abstract text
3. List of entities in the format: [Entity Names], Entity ID, Entity Type

Output Format:
Relation Type, Entity 1 ID, Entity 2 ID, Novelty

Entity Types:
- Gene: Genes, proteins, mRNA, and other gene products (NCBI Gene ID)
- Chemical: Chemicals and drugs (MeSH ID)
- Disease: Diseases, symptoms, and phenotypes (MeSH/OMIM ID)
- Variant: Genomic and protein variants (dbSNP ID or component representation)
- Species: Species names (NCBI Taxonomy ID)
- CellLine: Cell line names (Cellosaurus ID)

Relation Types:
- Association: Relation between two entities where the association is unclear
- Comparison: Comparison of effects or properties of two chemicals/drugs
- Conversion: Transformation of one chemical into another
- Cotreatment: Use of two or more chemicals/drugs as combination therapy
- Negative_Correlation: Inverse or opposing effect between two entities
- Positive_Correlation: Direct or reinforcing effect between two entities
- Bind: Physical interaction or binding between two entities
- Drug_Interaction: Pharmacological interaction between two co-administered drugs

Novelty:
- Novel: Relation related to the main point or novelty of the abstract
- No: Relation providing background information or context

Guidelines:
- Extract all relevant relations between the given entities from the abstract text
- Ensure the output follows the specified format exactly
- Each entiy pairs has only one relation. there should not be two relations with the same entities in results.

#Example Input1:
The differential effects of bupivacaine and lidocaine on prostaglandin E2 release, cyclooxygenase gene expression and pain in a clinical pain model.
BACKGROUND: In addition to blocking nociceptive input from surgical sites, long-acting local anesthetics might directly modulate inflammation. In the present study, we describe the proinflammatory effects of bupivacaine on local prostaglandin E2 (PGE2) production and cyclooxygenase (COX) gene expression that increases postoperative pain in human subjects. METHODS: Subjects (n = 114) undergoing extraction of impacted third molars received either 2% lidocaine or 0.5% bupivacaine before surgery and either rofecoxib 50 mg or placebo orally 90 min before surgery and for the following 48 h. Oral mucosal biopsies were taken before surgery and 48 h after surgery. After extraction, a microdialysis probe was placed at the surgical site for PGE2 and thromboxane B2 (TXB2) measurements. RESULTS: The bupivacaine/rofecoxib group reported significantly less pain, as assessed by a visual analog scale, compared with the other three treatment groups over the first 4 h. However, the bupivacaine/placebo group reported significantly more pain at 24 h and PGE2 levels during the first 4 h were significantly higher than the other three treatment groups. Moreover, bupivacaine significantly increased COX-2 gene expression at 48 h as compared with the lidocaine/placebo group. Thromboxane levels were not significantly affected by any of the treatments, indicating that the effects seen were attributable to inhibition of COX-2, but not COX-1. CONCLUSIONS: These results suggest that bupivacaine stimulates COX-2 gene expression after tissue injury, which is associated with higher PGE2 production and pain after the local anesthetic effect dissipates.

[bupivacaine],D002045,ChemicalEntity
[lidocaine],D008012,ChemicalEntity
[prostaglandin E2/ PGE2],5732,GeneOrGeneProduct
[cyclooxygenase/ COX],4512,4513,GeneOrGeneProduct
[pain],D010146,DiseaseOrPhenotypicFeature
[inflammation],D007249,DiseaseOrPhenotypicFeature
[postoperative pain],D010149,DiseaseOrPhenotypicFeature
[human],9606,OrganismTaxon
[rofecoxib],C116926,ChemicalEntity
[thromboxane B2/ TXB2],D013929,ChemicalEntity
[COX-2],4513,GeneOrGeneProduct
[Thromboxane],D013931,ChemicalEntity
[COX-1],4512,GeneOrGeneProduct
[tissue injury],D017695,DiseaseOrPhenotypicFeature

#Example Output1:
Association,4512,D010149,Novel
Negative_Correlation,D013931,4513,Novel
Association,4513,D010149,Novel
Positive_Correlation,4513,5732,Novel
Association,4513,D010146,Novel
Positive_Correlation,5732,D010149,Novel
Negative_Correlation,C116926,D010146,Novel
Positive_Correlation,D002045,D010149,Novel
Association,D002045,4512,Novel
Association,D002045,D017695,Novel
Positive_Correlation,D002045,4513,Novel
Positive_Correlation,D002045,5732,Novel
Positive_Correlation,D002045,D010146,Novel
Cotreatment,D002045,C116926,Novel
Comparison,D002045,D008012,Novel

#Example Input2:
H3 histamine receptor-mediated activation of protein kinase Calpha inhibits the growth of cholangiocarcinoma in vitro and in vivo.
Histamine regulates functions via four receptors (HRH1, HRH2, HRH3, and HRH4). The d-myo-inositol 1,4,5-trisphosphate (IP(3))/Ca(2+)/protein kinase C (PKC)/mitogen-activated protein kinase pathway regulates cholangiocarcinoma growth. We evaluated the role of HRH3 in the regulation of cholangiocarcinoma growth. Expression of HRH3 in intrahepatic and extrahepatic cell lines, normal cholangiocytes, and human tissue arrays was measured. In Mz-ChA-1 cells stimulated with (R)-(alpha)-(-)-methylhistamine dihydrobromide (RAMH), we measured (a) cell growth, (b) IP(3) and cyclic AMP levels, and (c) phosphorylation of PKC and mitogen-activated protein kinase isoforms. Localization of PKCalpha was visualized by immunofluorescence in cell smears and immunoblotting for PKCalpha in cytosol and membrane fractions. Following knockdown of PKCalpha, Mz-ChA-1 cells were stimulated with RAMH before evaluating cell growth and extracellular signal-regulated kinase (ERK)-1/2 phosphorylation. In vivo experiments were done in BALB/c nude mice. Mice were treated with saline or RAMH for 44 days and tumor volume was measured. Tumors were excised and evaluated for proliferation, apoptosis, and expression of PKCalpha, vascular endothelial growth factor (VEGF)-A, VEGF-C, VEGF receptor 2, and VEGF receptor 3. HRH3 expression was found in all cells. RAMH inhibited the growth of cholangiocarcinoma cells. RAMH increased IP(3) levels and PKCalpha phosphorylation and decreased ERK1/2 phosphorylation. RAMH induced a shift in the localization of PKCalpha expression from the cytosolic domain into the membrane region of Mz-ChA-1 cells. Silencing of PKCalpha prevented RAMH inhibition of Mz-ChA-1 cell growth and ablated RAMH effects on ERK1/2 phosphorylation. In vivo, RAMH decreased tumor growth and expression of VEGF and its receptors; PKCalpha expression was increased. RAMH inhibits cholangiocarcinoma growth by PKCalpha-dependent ERK1/2 dephosphorylation. Modulation of PKCalpha by histamine receptors may be important in regulating cholangiocarcinoma growth.

[H3 histamine receptor/ HRH3],11255,GeneOrGeneProduct
[protein kinase Calpha/ protein kinase C/ PKC/ PKCalpha],5578,GeneOrGeneProduct
[cholangiocarcinoma],D018281,DiseaseOrPhenotypicFeature
[Histamine],D006632,ChemicalEntity
[HRH1],3269,GeneOrGeneProduct
[HRH2],3274,GeneOrGeneProduct
[HRH4],59340,GeneOrGeneProduct
[d-myo-inositol 1,4,5-trisphosphate/ IP(3)],D015544,ChemicalEntity
[Ca(2+)],D002118,ChemicalEntity
[mitogen-activated protein kinase],5594,5595,GeneOrGeneProduct
[human],9606,OrganismTaxon
[Mz-ChA-1],CVCL_6932,CellLine
[(R)-(alpha)-(-)-methylhistamine dihydrobromide/ RAMH],C069357,ChemicalEntity
[cyclic AMP],D000242,ChemicalEntity
[extracellular signal-regulated kinase (ERK)-1/2/ ERK1/2],5595,5594,GeneOrGeneProduct
[BALB/c],CVCL_9101,CellLine
[mice],10090,OrganismTaxon
[tumor/ Tumors],D009369,DiseaseOrPhenotypicFeature
[vascular endothelial growth factor (VEGF)-A],7422,GeneOrGeneProduct
[VEGF-C],7424,GeneOrGeneProduct
[VEGF receptor 2],3791,GeneOrGeneProduct
[VEGF receptor 3],2324,GeneOrGeneProduct
[VEGF],7422,7424,GeneOrGeneProduct
[histamine receptors],11255,3269,3274,59340,GeneOrGeneProduct

#Example Output2:
Association,D002118,D018281,No
Association,D015544,D018281,No
Association,59340,D018281,Novel
Association,3274,D018281,Novel
Association,3269,D018281,Novel
Bind,D006632,59340,No
Bind,D006632,3274,No
Bind,D006632,3269,No
Bind,D006632,11255,No
Negative_Correlation,C069357,7424,Novel
Negative_Correlation,C069357,7422,Novel
Positive_Correlation,C069357,D015544,Novel
Negative_Correlation,C069357,D009369,Novel
Negative_Correlation,C069357,5595,Novel
Negative_Correlation,C069357,5594,Novel
Negative_Correlation,C069357,D018281,Novel
Association,5595,D018281,No
Association,5594,D018281,No
Association,5578,59340,Novel
Association,5578,3274,Novel
Association,5578,3269,Novel
Positive_Correlation,5578,C069357,Novel
Association,5578,5595,Novel
Association,5578,5594,Novel
Negative_Correlation,5578,D018281,No
Association,11255,D018281,Novel
Positive_Correlation,11255,5578,Novel

#Example Input3:
Sepsis-induced changes in amino acid transporters and leucine signaling via mTOR in skeletal muscle.
The present study tested the hypothesis that sepsis-induced leucine (Leu) resistance in skeletal muscle is associated with a down-regulation of amino acid transporters important in regulating Leu flux or an impairment in the formation of the Leu-sensitive mTOR-Ragulator complex. Sepsis in adult male rats decreased basal protein synthesis in gastrocnemius, associated with a reduction in mTOR activation as indicated by decreased 4E-BP1 and S6K1 phosphorylation. The ability of oral Leu to increase protein synthesis and mTOR kinase after 1 h was largely prevented in sepsis. Sepsis increased CAT1, LAT2 and SNAT2 mRNA content two- to fourfold, but only the protein content for CAT1 (20 % decrease) differed significantly. Conversely, sepsis decreased the proton-assisted amino acid transporter (PAT)-2 mRNA by 60 %, but without a coordinate change in PAT2 protein. There was no sepsis or Leu effect on the protein content for RagA-D, LAMTOR-1 and -2, raptor, Rheb or mTOR in muscle. The binding of mTOR, PRAS40 and RagC to raptor did not differ for control and septic muscle in the basal condition; however, the Leu-induced decrease in PRAS40 raptor and increase in RagC raptor seen in control muscle was absent in sepsis. The intracellular Leu concentration was increased in septic muscle, compared to basal control conditions, and oral Leu further increased the intracellular Leu concentration similarly in both control and septic rats. Hence, while alterations in select amino acid transporters are not associated with development of sepsis-induced Leu resistance, the Leu-stimulated binding of raptor with RagC and the recruitment of mTOR/raptor to the endosome-lysosomal compartment may partially explain the inability of Leu to fully activate mTOR and muscle protein synthesis.

[Sepsis],D018805,DiseaseOrPhenotypicFeature
[amino acid transporters],246235,25648,29642,84551,GeneOrGeneProduct
[leucine/ Leu],D007930,ChemicalEntity
[mTOR],56718,GeneOrGeneProduct
[Ragulator complex/ LAMTOR-1 and -2],295234,308869,GeneOrGeneProduct
[rats],10116,OrganismTaxon
[4E-BP1],116636,GeneOrGeneProduct
[S6K1],83840,GeneOrGeneProduct
[CAT1],25648,GeneOrGeneProduct
[LAT2],84551,GeneOrGeneProduct
[SNAT2],29642,GeneOrGeneProduct
[proton-assisted amino acid transporter (PAT)-2/ PAT2],246235,GeneOrGeneProduct
[RagA-D],117043,117044,297960,298514,GeneOrGeneProduct
[raptor],287871,GeneOrGeneProduct
[Rheb],26954,GeneOrGeneProduct
[PRAS40],292887,GeneOrGeneProduct
[RagC],298514,GeneOrGeneProduct

#Example Output3:
Bind,292887,287871,No
Bind,287871,298514,No
Association,D007930,308869,No
Association,D007930,295234,No
Positive_Correlation,D007930,298514,Novel
Negative_Correlation,D007930,292887,Novel
Association,D007930,287871,Novel
Association,D007930,D018805,Novel
Positive_Correlation,D007930,56718,Novel
Association,D018805,29642,Novel
Association,D018805,84551,Novel
Association,D018805,25648,Novel
Negative_Correlation,D018805,56718,Novel
Association,56718,308869,No
Association,56718,295234,No
Bind,56718,287871,No
Association,56718,83840,Novel
Association,56718,116636,Novel

#Example Input4:
Glucose metabolism in patients with schizophrenia treated with atypical antipsychotic agents: a frequently sampled intravenous glucose tolerance test and minimal model analysis.
BACKGROUND: While the incidence of new-onset diabetes mellitus may be increasing in patients with schizophrenia treated with certain atypical antipsychotic agents, it remains unclear whether atypical agents are directly affecting glucose metabolism or simply increasing known risk factors for diabetes. OBJECTIVE: To study the 2 drugs most clearly implicated (clozapine and olanzapine) and risperidone using a frequently sampled intravenous glucose tolerance test. DESIGN: A cross-sectional design in stable, treated patients with schizophrenia evaluated using a frequently sampled intravenous glucose tolerance test and the Bergman minimal model analysis. SETTING: Subjects were recruited from an urban community mental health clinic and were studied at a general clinical research center. Patients Fifty subjects signed informed consent and 41 underwent the frequently sampled intravenous glucose tolerance test. Thirty-six nonobese subjects with schizophrenia or schizoaffective disorder, matched by body mass index and treated with either clozapine, olanzapine, or risperidone, were included in the analysis. MAIN OUTCOME MEASURES: Fasting plasma glucose and fasting serum insulin levels, insulin sensitivity index, homeostasis model assessment of insulin resistance, and glucose effectiveness. RESULTS: The mean +/- SD duration of treatment with the identified atypical antipsychotic agent was 68.3 +/- 28.9 months (clozapine), 29.5 +/- 17.5 months (olanzapine), and 40.9 +/- 33.7 (risperidone). Fasting serum insulin concentrations differed among groups (F(33) = 3.35; P = .047) (clozapine>olanzapine>risperidone) with significant differences between clozapine and risperidone (t(33) = 2.32; P = .03) and olanzapine and risperidone (t(33) = 2.15; P = .04). There was a significant difference in insulin sensitivity index among groups (F(33) = 10.66; P<.001) (clozapine<olanzapine<risperidone), with subjects who received clozapine and olanzapine exhibiting significant insulin resistance compared with subjects who were treated with risperidone (clozapine vs risperidone, t(33) = -4.29; P<.001; olanzapine vs risperidone, t(33) = -3.62; P = .001 [P<.001]). The homeostasis model assessment of insulin resistance also differed significantly among groups (F(33) = 4.92; P = .01) (clozapine>olanzapine>risperidone) (clozapine vs risperidone, t(33) = 2.94; P = .006; olanzapine vs risperidone, t(33) = 2.42; P = .02). There was a significant difference among groups in glucose effectiveness (F(30) = 4.18; P = .02) (clozapine<olanzapine<risperidone) with significant differences between clozapine and risperidone (t(30) = -2.59; P = .02) and olanzapine and risperidone (t(30) = -2.34, P = .03). CONCLUSIONS: Both nonobese clozapine- and olanzapine-treated groups displayed significant insulin resistance and impairment of glucose effectiveness compared with risperidone-treated subjects. Patients taking clozapine and olanzapine must be examined for insulin resistance and its consequences.

[Glucose],D005947,ChemicalEntity
[patients],9606,OrganismTaxon
[schizophrenia],D012559,DiseaseOrPhenotypicFeature
[antipsychotic agents/ antipsychotic agent],D014150,ChemicalEntity
[diabetes mellitus/ diabetes],D003920,DiseaseOrPhenotypicFeature
[clozapine],D003024,ChemicalEntity
[olanzapine],C076029,ChemicalEntity
[risperidone],D018967,ChemicalEntity
[schizoaffective disorder],D011618,DiseaseOrPhenotypicFeature
[insulin],D007328,ChemicalEntity
[insulin resistance],D007333,DiseaseOrPhenotypicFeature

#Example Output4:
Negative_Correlation,D011618,D018967,No
Negative_Correlation,D011618,C076029,No
Negative_Correlation,D011618,D003024,No
Association,D018967,D005947,Novel
Positive_Correlation,C076029,D007333,Novel
Comparison,C076029,D018967,Novel
Association,C076029,D005947,Novel
Positive_Correlation,D003024,D007333,Novel
Comparison,D003024,D018967,Novel
Comparison,D003024,C076029,Novel
Association,D003024,D005947,Novel
Negative_Correlation,D012559,D018967,No
Negative_Correlation,D012559,C076029,No
Negative_Correlation,D012559,D003024,No
Negative_Correlation,D012559,D014150,No
Positive_Correlation,D014150,D003920,Novel
Association,D005947,D014150,Novel

#Example Input5:
Genetic variation in the COX-2 gene and the association with prostate cancer risk.
COX-2 is a key enzyme in the conversion of arachidonic acid to prostaglandins. The prostaglandins produced by COX-2 are involved in inflammation and pain response in different tissues in the body. Accumulating evidence from epidemiologic studies, chemical carcinogen-induced rodent models and clinical trials indicate that COX-2 plays a role in human carcinogenesis and is overexpressed in prostate cancer tissue. We examined whether sequence variants in the COX-2 gene are associated with prostate cancer risk. We analyzed a large population-based case-control study, cancer prostate in Sweden (CAPS) consisting of 1,378 cases and 782 controls. We evaluated 16 single nucleotide polymorphisms (SNPs) spanning the entire COX-2 gene in 94 subjects of the control group. Five SNPs had a minor allele frequency of more than 5% in our study population and these were genotyped in all case patients and control subjects and gene-specific haplotypes were constructed. A statistically significant difference in allele frequency between cases and controls was observed for 2 of the SNPs (+3100 T/G and +8365 C/T), with an odds ratio of 0.78 (95% CI=0.64-0.96) and 0.65 (95% CI=0.45-0.94) respectively. In the haplotype analysis, 1 haplotype carrying the variant allele from both +3100 T/G and +8365 C/T, with a population frequency of 3%, was also significantly associated with decreased risk of prostate cancer (p=0.036, global simulated p-value=0.046). This study supports the hypothesis that inflammation is involved in prostate carcinogenesis and that sequence variation within the COX-2 gene influence the risk of prostate cancer.

[COX-2],4513,GeneOrGeneProduct
[prostate cancer/ cancer prostate/ prostate carcinogenesis],D011471,DiseaseOrPhenotypicFeature
[arachidonic acid],D016718,ChemicalEntity
[prostaglandins],D011453,ChemicalEntity
[inflammation],D007249,DiseaseOrPhenotypicFeature
[pain],D010146,DiseaseOrPhenotypicFeature
[human/ patients],9606,OrganismTaxon
[carcinogenesis],D009369,DiseaseOrPhenotypicFeature
[+3100 T/G],c|SUB|T|+3100|G,SequenceVariant
[+8365 C/T],c|SUB|C|+8365|T,SequenceVariant

#Example Output5:
Negative_Correlation,c|SUB|C|+8365|T,D011471,Novel
Association,c|SUB|T|+3100|G,c|SUB|C|+8365|T,Novel
Negative_Correlation,c|SUB|T|+3100|G,D011471,Novel
Association,D011453,D010146,No
Association,D011453,D007249,No
Conversion,D016718,D011453,No
Association,4513,D010146,No
Association,4513,D007249,No
Positive_Correlation,4513,D011453,No
Association,4513,D016718,No
Positive_Correlation,4513,D011471,Novel
Association,4513,D009369,No
'''

In [7]:
# @title p init
p_init = """You are an AI assistant specializing in extracting biomedical relations from scientific articles. Your task is to identify the biomedical entity pairs with relations in the given text and output them in a structured format.

Input Format:
1. Title of the article
2. Abstract text
3. List of entities in the format: [Entity Names], Entity ID, Entity Type

Entity Types:
- Gene: Genes, proteins, mRNA, and other gene products (NCBI Gene ID)
- Chemical: Chemicals and drugs (MeSH ID)
- Disease: Diseases, symptoms, and phenotypes (MeSH/OMIM ID)
- Variant: Genomic and protein variants (dbSNP ID or component representation)
- Species: Species names (NCBI Taxonomy ID)
- CellLine: Cell line names (Cellosaurus ID)

Output Format:
Entity 1 ID, Entity 2 ID

#Example Input
Genetic variation in the COX-2 gene and the association with prostate cancer risk.
COX-2 is a key enzyme in the conversion of arachidonic acid to prostaglandins. The prostaglandins produced by COX-2 are involved in inflammation and pain response in different tissues in the body. Accumulating evidence from epidemiologic studies, chemical carcinogen-induced rodent models and clinical trials indicate that COX-2 plays a role in human carcinogenesis and is overexpressed in prostate cancer tissue. We examined whether sequence variants in the COX-2 gene are associated with prostate cancer risk. We analyzed a large population-based case-control study, cancer prostate in Sweden (CAPS) consisting of 1,378 cases and 782 controls. We evaluated 16 single nucleotide polymorphisms (SNPs) spanning the entire COX-2 gene in 94 subjects of the control group. Five SNPs had a minor allele frequency of more than 5% in our study population and these were genotyped in all case patients and control subjects and gene-specific haplotypes were constructed. A statistically significant difference in allele frequency between cases and controls was observed for 2 of the SNPs (+3100 T/G and +8365 C/T), with an odds ratio of 0.78 (95% CI=0.64-0.96) and 0.65 (95% CI=0.45-0.94) respectively. In the haplotype analysis, 1 haplotype carrying the variant allele from both +3100 T/G and +8365 C/T, with a population frequency of 3%, was also significantly associated with decreased risk of prostate cancer (p=0.036, global simulated p-value=0.046). This study supports the hypothesis that inflammation is involved in prostate carcinogenesis and that sequence variation within the COX-2 gene influence the risk of prostate cancer.

[COX-2],4513,GeneOrGeneProduct
[prostate cancer/ cancer prostate/ prostate carcinogenesis],D011471,DiseaseOrPhenotypicFeature
[arachidonic acid],D016718,ChemicalEntity
[prostaglandins],D011453,ChemicalEntity
[inflammation],D007249,DiseaseOrPhenotypicFeature
[pain],D010146,DiseaseOrPhenotypicFeature
[human/ patients],9606,OrganismTaxon
[carcinogenesis],D009369,DiseaseOrPhenotypicFeature
[+3100 T/G],c|SUB|T|+3100|G,SequenceVariant
[+8365 C/T],c|SUB|C|+8365|T,SequenceVariant

#Example Output
c|SUB|C|+8365|T,D011471
c|SUB|T|+3100|G,c|SUB|C|+8365|T
c|SUB|T|+3100|G,D011471
D011453,D010146
D011453,D007249
D016718,D011453
4513,D010146
4513,D007249
4513,D011453
4513,D016718
4513,D011471
4513,D009369
"""

In [ ]:
# @title idk
'''
#Example Input1
The differential effects of bupivacaine and lidocaine on prostaglandin E2 release, cyclooxygenase gene expression and pain in a clinical pain model.
BACKGROUND: In addition to blocking nociceptive input from surgical sites, long-acting local anesthetics might directly modulate inflammation. In the present study, we describe the proinflammatory effects of bupivacaine on local prostaglandin E2 (PGE2) production and cyclooxygenase (COX) gene expression that increases postoperative pain in human subjects. METHODS: Subjects (n = 114) undergoing extraction of impacted third molars received either 2% lidocaine or 0.5% bupivacaine before surgery and either rofecoxib 50 mg or placebo orally 90 min before surgery and for the following 48 h. Oral mucosal biopsies were taken before surgery and 48 h after surgery. After extraction, a microdialysis probe was placed at the surgical site for PGE2 and thromboxane B2 (TXB2) measurements. RESULTS: The bupivacaine/rofecoxib group reported significantly less pain, as assessed by a visual analog scale, compared with the other three treatment groups over the first 4 h. However, the bupivacaine/placebo group reported significantly more pain at 24 h and PGE2 levels during the first 4 h were significantly higher than the other three treatment groups. Moreover, bupivacaine significantly increased COX-2 gene expression at 48 h as compared with the lidocaine/placebo group. Thromboxane levels were not significantly affected by any of the treatments, indicating that the effects seen were attributable to inhibition of COX-2, but not COX-1. CONCLUSIONS: These results suggest that bupivacaine stimulates COX-2 gene expression after tissue injury, which is associated with higher PGE2 production and pain after the local anesthetic effect dissipates.

[bupivacaine],D002045,ChemicalEntity
[lidocaine],D008012,ChemicalEntity
[prostaglandin E2/ PGE2],5732,GeneOrGeneProduct
[cyclooxygenase/ COX],4512,4513,GeneOrGeneProduct
[pain],D010146,DiseaseOrPhenotypicFeature
[inflammation],D007249,DiseaseOrPhenotypicFeature
[postoperative pain],D010149,DiseaseOrPhenotypicFeature
[human],9606,OrganismTaxon
[rofecoxib],C116926,ChemicalEntity
[thromboxane B2/ TXB2],D013929,ChemicalEntity
[COX-2],4513,GeneOrGeneProduct
[Thromboxane],D013931,ChemicalEntity
[COX-1],4512,GeneOrGeneProduct
[tissue injury],D017695,DiseaseOrPhenotypicFeature

#Example Output1
4512,D010149
D013931,4513
4513,D010149
4513,5732
4513,D010146
5732,D010149
C116926,D010146
D002045,D010149
D002045,4512
D002045,D017695
D002045,4513
D002045,5732
D002045,D010146
D002045,C116926
D002045,D008012

#Example Input2
H3 histamine receptor-mediated activation of protein kinase Calpha inhibits the growth of cholangiocarcinoma in vitro and in vivo.
Histamine regulates functions via four receptors (HRH1, HRH2, HRH3, and HRH4). The d-myo-inositol 1,4,5-trisphosphate (IP(3))/Ca(2+)/protein kinase C (PKC)/mitogen-activated protein kinase pathway regulates cholangiocarcinoma growth. We evaluated the role of HRH3 in the regulation of cholangiocarcinoma growth. Expression of HRH3 in intrahepatic and extrahepatic cell lines, normal cholangiocytes, and human tissue arrays was measured. In Mz-ChA-1 cells stimulated with (R)-(alpha)-(-)-methylhistamine dihydrobromide (RAMH), we measured (a) cell growth, (b) IP(3) and cyclic AMP levels, and (c) phosphorylation of PKC and mitogen-activated protein kinase isoforms. Localization of PKCalpha was visualized by immunofluorescence in cell smears and immunoblotting for PKCalpha in cytosol and membrane fractions. Following knockdown of PKCalpha, Mz-ChA-1 cells were stimulated with RAMH before evaluating cell growth and extracellular signal-regulated kinase (ERK)-1/2 phosphorylation. In vivo experiments were done in BALB/c nude mice. Mice were treated with saline or RAMH for 44 days and tumor volume was measured. Tumors were excised and evaluated for proliferation, apoptosis, and expression of PKCalpha, vascular endothelial growth factor (VEGF)-A, VEGF-C, VEGF receptor 2, and VEGF receptor 3. HRH3 expression was found in all cells. RAMH inhibited the growth of cholangiocarcinoma cells. RAMH increased IP(3) levels and PKCalpha phosphorylation and decreased ERK1/2 phosphorylation. RAMH induced a shift in the localization of PKCalpha expression from the cytosolic domain into the membrane region of Mz-ChA-1 cells. Silencing of PKCalpha prevented RAMH inhibition of Mz-ChA-1 cell growth and ablated RAMH effects on ERK1/2 phosphorylation. In vivo, RAMH decreased tumor growth and expression of VEGF and its receptors; PKCalpha expression was increased. RAMH inhibits cholangiocarcinoma growth by PKCalpha-dependent ERK1/2 dephosphorylation. Modulation of PKCalpha by histamine receptors may be important in regulating cholangiocarcinoma growth.

[H3 histamine receptor/ HRH3],11255,GeneOrGeneProduct
[protein kinase Calpha/ protein kinase C/ PKC/ PKCalpha],5578,GeneOrGeneProduct
[cholangiocarcinoma],D018281,DiseaseOrPhenotypicFeature
[Histamine],D006632,ChemicalEntity
[HRH1],3269,GeneOrGeneProduct
[HRH2],3274,GeneOrGeneProduct
[HRH4],59340,GeneOrGeneProduct
[d-myo-inositol 1,4,5-trisphosphate/ IP(3)],D015544,ChemicalEntity
[Ca(2+)],D002118,ChemicalEntity
[mitogen-activated protein kinase],5594,5595,GeneOrGeneProduct
[human],9606,OrganismTaxon
[Mz-ChA-1],CVCL_6932,CellLine
[(R)-(alpha)-(-)-methylhistamine dihydrobromide/ RAMH],C069357,ChemicalEntity
[cyclic AMP],D000242,ChemicalEntity
[extracellular signal-regulated kinase (ERK)-1/2/ ERK1/2],5595,5594,GeneOrGeneProduct
[BALB/c],CVCL_9101,CellLine
[mice],10090,OrganismTaxon
[tumor/ Tumors],D009369,DiseaseOrPhenotypicFeature
[vascular endothelial growth factor (VEGF)-A],7422,GeneOrGeneProduct
[VEGF-C],7424,GeneOrGeneProduct
[VEGF receptor 2],3791,GeneOrGeneProduct
[VEGF receptor 3],2324,GeneOrGeneProduct
[VEGF],7422,7424,GeneOrGeneProduct
[histamine receptors],11255,3269,3274,59340,GeneOrGeneProduct

#Example Output2
D002118,D018281
D015544,D018281
59340,D018281
3274,D018281
3269,D018281
D006632,59340
D006632,3274
D006632,3269
D006632,11255
C069357,7424
C069357,7422
C069357,D015544
C069357,D009369
C069357,5595
C069357,5594
C069357,D018281
5595,D018281
5594,D018281
5578,59340
5578,3274
5578,3269
5578,C069357
5578,5595
5578,5594
5578,D018281
11255,D018281
11255,5578
'''

In [8]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models



generation_config = {
    "temperature": 0,
    "max_output_tokens": 2048,
    "top_p": 1,
    "top_k": 1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

# pairs
vertexai.init(project="439299698879", location="us-central1")
model = GenerativeModel(
  "projects/439299698879/locations/us-central1/endpoints/6166859454009573376",
  # system_instruction=[textsi_1],
  generation_config=generation_config,
  safety_settings=safety_settings)

# # all pairs + type + novelty
# vertexai.init(project="439299698879", location="us-central1")
# model = GenerativeModel(
#   "projects/439299698879/locations/us-central1/endpoints/7839805978089357312",
#   # system_instruction=[textsi_2],
#   generation_config=generation_config,
#   safety_settings=safety_settings
# )

# #pairs no instruction
# vertexai.init(project="439299698879", location="us-central1")
# model = GenerativeModel(
#   "projects/439299698879/locations/us-central1/endpoints/4720218811205156864",
#   generation_config=generation_config,
#   safety_settings=safety_settings
# )



In [9]:
types = ['Association', 'Comparison', 'Conversion', 'Cotreatment', 'Negative_Correlation', 'Positive_Correlation', 'Bind', 'Drug_Interaction']
doc , RDP, Tokens =0,0,0

In [10]:
def prompt_model(i):
  global Tokens
  global c

  pmid = new_test[str(i)]["pmid"]
  print(i, "." , pmid)
  # prompt = add_exaples_prompt(init_prompt)
  prompt = p_init
  prompt += "produce similar output for this article:" + '\n'
  # prompt = "identify the entity pairs with relations in this article:" + '\n'
  prompt += inputs[i]
  try:
    response = model.generate_content(prompt)
    Tokens += len(prompt.split())
    c+=1

  except:
    print("no response try again:", pmid)
    response = model.generate_content(prompt)
    Tokens += len(prompt.split())
    c+=1

  return response.text


def correct_format(result):
  try:
    first_par = result.split('\n')[0].split(',')[0]
    if first_par in types:
      return True
    else:
      return False
  except:
    return False

def create_inputs():
  inputs = []
  for i in range(len(new_test)):
    example = ""
    doc = new_test[str(i)]
    ents = ""
    for e in doc["entities"]:
      ents += '[' + "/ ".join(e["names"]) + ']' + "," + e["id"]+ ","  + e["type"] + "\n"

    # example += "pmid:" + doc["pmid"] +"\n"
    example += doc["title"] +"\n"
    example += doc["article"] +"\n"
    example +=  ents +"\n"
    i += 1
    inputs.append(example)
  return inputs

In [11]:
inputs = create_inputs()

In [ ]:
print(len(inputs))

100


In [12]:
print(doc,Tokens, RDP)

0 0 0


In [41]:
doc, Tokens, RDP = 0,0,0

In [17]:

blocked_doc=[74, 79]
# blocked_doc_noins = [22,92]


In [23]:
doc =80

In [24]:
%time
while (doc < 100 and RDP < 1500):
  c=0
  Tokens = 0

  while(Tokens < 18000  and doc < 100):
    outputs = ""
    pmid = new_test[str(doc)]["pmid"]
    result = prompt_model(doc)
    if len(result) > 0:
      outputs += "pmid:" + pmid+ "\n" + result + "\n\n"
      doc +=1
      with open("/content/tuned_pair_model_1_shot.txt", "a") as f:
        f.write(outputs)
    else:
        print(doc, "." , pmid, "try again")



  print("last doc: ", doc)
  print("tokens:", Tokens)

  time.sleep(60)


  # time.sleep(6)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
80 . 27384305
81 . 15000256
82 . 15811908
83 . 19957053
84 . 17935240
85 . 18441470
86 . 15036754
87 . 15673851
88 . 15987266
89 . 16428827
90 . 16844102
91 . 17562951
92 . 17879945
93 . 18399341
94 . 18439803
95 . 19531695
96 . 24114426
97 . 24717468
98 . 24739405
99 . 24840785
last doc:  100
tokens: 13394


In [ ]:
doc = 92
pmid = new_test[str(doc)]["pmid"]
result = prompt_model(doc)
print(result)

92 . 17879945


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "OTHER"
}
Response:
{
  "candidates": [
    {
      "finish_reason": "OTHER"
    }
  ],
  "usage_metadata": {
    "prompt_token_count": 670,
    "total_token_count": 670
  }
}

In [ ]:
doc = 66
pmid = new_test[str(doc)]["pmid"]
result = prompt_model(doc)
print(result)

66 . 19484664
Association,D001249,1636,Novel
Association,D004342,1636,No
Association,D001102,1636,No



In [ ]:
doc = 84
pmid = new_test[str(doc)]["pmid"]
result = prompt_model(doc)
print(result)

84 . 17935240
Association,D012174,g|DEL|ORF15+652_653|AG,No
Association,D012174,6103,No
Association,g|DEL|ORF15+652_653|AG,6103,No



In [ ]:
pmid = new_test[str(21)]["pmid"]
print(pmid)

22369755


##test

In [ ]:
prompt = '''
Congenital hypothyroidism due to a new deletion in the sodium/iodide symporter protein.
OBJECTIVE: Iodide transport defect (ITD) is a rare disorder characterised by an inability of the thyroid to maintain an iodide gradient across the basolateral membrane of thyroid follicular cells, that often results in congenital hypothyroidism. When present the defect is also found in the salivary glands and gastric mucosa and it has been shown to arise from abnormalities of the sodium/iodide symporter (NIS). PATIENT: We describe a woman with hypothyroidism identified at the 3rd month of life. The diagnosis of ITD was suspected because of nodular goitre, and little if any iodide uptake by the thyroid and salivary glands. Treatment with iodide partially corrected the hypothyroidism; however, long-term substitution therapy with L-thyroxine was started. MEASUREMENTS: Thyroid radioiodide uptake was only 1.4% and 0.3% at 1 and 24 h after the administration of recombinant human TSH. The saliva to plasma I- ratio was 1.1 indicating that the inability of the thyroid gland to concentrate I- was also present in the salivary glands. RESULTS: Analysis of the patient's NIS gene revealed a 15 nucleotide (nt) deletion of the coding sequence (nt 1314 through nt 1328) and the insertion of 15 nt duplicating the first 15 nt of the adjacent intron. The patient was homozygous for this insertion/deletion, while both consanguineous parents were heterozygous. This deletion predicts the production of a protein lacking the five terminal amino acids of exon XI (439-443) which are located in the 6th intracellular loop. COS-7 cells transfected with a vector expressing the mutant del-(439-443) NIS failed to concentrate iodide, suggesting that the mutation was the direct cause of the ITD in this patient. CONCLUSION: In conclusion we describe the first Italian case of congenital hypothyroidism due to a new deletion in the NIS gene.
[Congenital hypothyroidism],D003409,DiseaseOrPhenotypicFeature
[sodium/iodide symporter/ NIS],6528,GeneOrGeneProduct
[Iodide transport defect/ ITD],C564766,DiseaseOrPhenotypicFeature
[inability of the thyroid],D050033,DiseaseOrPhenotypicFeature
[iodide/ radioiodide/ I-],D007454,ChemicalEntity
[PATIENT/ woman/ human],9606,OrganismTaxon
[hypothyroidism],D007037,DiseaseOrPhenotypicFeature
[nodular goitre],D020518,DiseaseOrPhenotypicFeature
[L-thyroxine],D013974,ChemicalEntity
[TSH],7200,GeneOrGeneProduct
[deletion of the coding sequence (nt 1314 through nt 1328)],c|DEL|1314_1328|,SequenceVariant
[insertion of 15 nt duplicating the first 15],c|DUP||15,SequenceVariant
[lacking the five terminal amino acids of exon XI (439-443)/ del-(439-443)],p|DEL|439_443|,SequenceVariant
[COS-7],CVCL_0224,CellLine
'''

In [ ]:
vertexai.init(project="439299698879", location="us-central1")
model = GenerativeModel(
  "projects/439299698879/locations/us-central1/endpoints/6166859454009573376",
  system_instruction=[textsi_1],
  generation_config=generation_config,
  safety_settings=safety_settings)
response = model.generate_content(prompt)
print(response.text)

D007454,C564766
D007454,D050033
D007454,D003409
C564766,D003409
C564766,6528
6528,D007454
6528,D003409
6528,C564766

